In [0]:
storage_account = "rqstgacct"
container = "inputsource"
file_path = "2015-summary.csv"
access_key = "h3eQACUU/LPjCmFcxlCDjW6+SJ0VuA6xHSAQpH3B+JJtVfQeV7si9sSu5SclIvppU/2NktZpDPna+AStvpzm/g=="


In [0]:
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    access_key
)


In [0]:
flightData2015  = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"wasbs://{container}@{storage_account}.blob.core.windows.net/{file_path}")
flightData2015 .show()


+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [0]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonSort [count#65 ASC NULLS FIRST]
         +- PhotonShuffleExchangeSource
            +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#125]
               +- PhotonShuffleExchangeSink rangepartitioning(count#65 ASC NULLS FIRST, 200)
                  +- PhotonRowToColumnar
                     +- FileScan csv [DEST_COUNTRY_NAME#63,ORIGIN_COUNTRY_NAME#64,count#65] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[wasbs://inputsource@rqstgacct.blob.core.windows.net/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


== Photon Explanation ==
The query is fully supported by Photon.


In [0]:
 spark.conf.set("spark.sql.shuffle.partitions", "5")

In [0]:
 flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)]

In [0]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [0]:
sqlWay = spark.sql("""
 SELECT DEST_COUNTRY_NAME, count(1)
 FROM flight_data_2015
 GROUP BY DEST_COUNTRY_NAME
 """)
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonGroupingAgg(keys=[DEST_COUNTRY_NAME#63], functions=[finalmerge_count(merge count#101L) AS count(1)#98L])
         +- PhotonShuffleExchangeSource
            +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#196]
               +- PhotonShuffleExchangeSink hashpartitioning(DEST_COUNTRY_NAME#63, 5)
                  +- PhotonGroupingAgg(keys=[DEST_COUNTRY_NAME#63], functions=[partial_count(1) AS count#101L])
                     +- PhotonRowToColumnar
                        +- FileScan csv [DEST_COUNTRY_NAME#63] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[wasbs://inputsource@rqstgacct.blob.core.windows.net/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Photon Explanation ==
The query is fully supported by Photon.


In [0]:
dataFrameWay = flightData2015\
 .groupBy("DEST_COUNTRY_NAME")\
 .count()

dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonGroupingAgg(keys=[DEST_COUNTRY_NAME#63], functions=[finalmerge_count(merge count#109L) AS count(1)#107L])
         +- PhotonShuffleExchangeSource
            +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#238]
               +- PhotonShuffleExchangeSink hashpartitioning(DEST_COUNTRY_NAME#63, 5)
                  +- PhotonGroupingAgg(keys=[DEST_COUNTRY_NAME#63], functions=[partial_count(1) AS count#109L])
                     +- PhotonRowToColumnar
                        +- FileScan csv [DEST_COUNTRY_NAME#63] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[wasbs://inputsource@rqstgacct.blob.core.windows.net/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Photon Explanation ==
The query is fully supported by Photon.


In [0]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

